# Lab  - LSTM for Air Polution Forecasting



**Objectives**: Apply Long Short-Term Memory (LSTM) Recurrent Neural Network (RNN) for 
Multivariate Time Series Forecasting. 

**The forecasting problem**: Air Quality dataset reports on the weather and the level of pollution each hour over 5 years at the US embassy in Beijing, China. Given the weather conditions and pollution for prior hours, forecast the pollution at the next hour.

Data includes the date-time, the pollution called PM2.5 concentration, and the weather information including dew point, temperature, pressure, wind direction, wind speed and the cumulative number of hours of snow and rain. The complete list of data is as follows:

**Total data**: 43800 samples (5 years x 365 days x 24 hours), 8 features.

**year-month-day-hour**: date of data in this row

**pm2.5**: PM2.5 concentration  (**pollution indicator => the predicted variable**) 

**DEWP**: Dew Point (DewP indicates the amount moisture in the air. The higher the dewP, the higher the moisture content of the air at a given temperature.

**TEM**P: Temperature

**PRES**: Pressure

**cbwd**: Combined wind direction (categorical feature, 4 values, e.g. Sought-East (SE))

**Iws**: Cumulated wind speed

**Is**: Cumulated hours of snow

**Ir**: Cumulated hours of rain

In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=RuntimeWarning)

import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

## Load Data
Use function *read_csv* to load data from file *pollution.csv*. 
This function will create dataframe dataset. 

Dataframe consists not only of values but also 
the names of the columns, the indices of the rows. 

In [2]:

dataset =  ?

#Print the dimension of dataset 
?

#Print the first few lines of the structure dataset 
?

#Print only the names of the columns
?

n_features =  ?  #number of features 

# Extract only the values of dataframe and ignore the column with the date. 
values = ?

SyntaxError: invalid syntax (Temp/ipykernel_12868/1156196339.py, line 1)

## Plot Data
Plot the data and get a figure similar to Fig.1 
<img src="images/f1.png" style="width:350px;height:250px;">
<caption><center> **Fig. 1** : ** Pollution & Weather Time series dataset** </center></caption>

In [ ]:
##use for example pyplot

?


## LSTM Data Preparation
Prepare the dataset for the LSTM, that is, frame the dataset as a supervised learning problem to predict the pollution (pm2.5) at the current or future hours given the past pollution and weather conditions over a number of prior time steps (hours).


**Original data frame (each row has data of only that hour)** :

row 1=> var1(t), var2(t),...var8(t)

row 2=> var1(t-1), var2(t-1),...var8(t)

row 3=> .....

**After series_to_supervised (each row has data of n previous hours and the current hour)**:


row 1=> var1(t-n), var2(t-n),...var8(t-n), var1(t-n+1), var2(t-n+1),...var8(t-n+1), .....var1(t), var3(t),...var8(t)

row 2=> var1(t-n-1), var2(t-n-1),...var8(t-n-1), var1(t-1), var3(t-1),...var8(t-1)

row 3 => ...




In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True): 
    """
        data: data matrix
        n_in: number of input lag timesteps (hours)
        n_out: number of output prediction timesteps (hours)
 
    """
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)  #transforms data (pure matrix of values) into data structure adding index of rows and columns
    cols, names = list(), list()  #inicialize empty lists 
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 

In [ ]:
# encode the categorical feature cbwd: Combined wind direction (4 categorical values) into integers (0,1,2,3)

encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4]) # transform the 5th column into integers 0,1,2,3

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled =  ?  #apply scaler to the values 


# specify the number of lag hours. You can change this hyper-parameter
lag_hours = 3

# call series_to_supervised to frame data as supervised learning
reframed = ?

#Print the dimension of  the dataframe reframed: ANSWER=(43797, 32) 
?

#Print the first few lines of reframed
?

# we want to predict only the feature pm2.5 =var1(t)
# Take out the columns of var2(t), var3(t),...var8(t).
reframed.drop(?)

#Print the new dimension of  reframed: ANSWER=(43797, 25) 
?

#Print the first few lines of the new reframed
?


## Split Data

First, data is split into train and test subsets. To speed up the model fiting, the training data is only the 
1st year, the test data are the remaining 4 years. You may consider exploring the oposite. 

Then the train and test sets are split into input and output variables. 

Finally, the inputs (X) are reshaped into 3D format expected by LSTMs, namely [samples, timesteps, features].

**Total data**: 43800 samples (5 years x 365 days x 24 hours), 8 features.

**Train data**: 3D tensor(samples, timesteps, features).
8760 samples:  Data from 1 year (365 days), 24 times per day (1h period).

**Test data**: 3D tensor (samples, timesteps, features). The rest of 35037 samples. 

In [ ]:
# split into train and test sets

# reframed is a dataframe that has not only values but also heads and row indixes. 
#Extract only the values 

values = reframed.values
n_train_hours = 365 * 24  #8760 (1 year, 24 times per day, i.e. period 1h)

#Extract training subset from values 
train = ?

#Extract test subset from values 
test = ?

train_X  = ?   # train_X.shape = (8760, 24)
train_y = ?  # train_y.shape = (8760,)

test_X  =  ?      # test_X.shape = (35037, 24)
test_y =  ? # test_y.shape = (35037,)

# reshape input to be 3D [samples, lag_hours, features]
train_X = ? # train_X.shape=(8760,3,8)
test_X =  ?  # test_X.shape=(35037,3,8)


## Define and Fit Model

In this section, we define and fit an LSTM on the multivariate input data.
We define the LSTM with 50 neurons in the first hidden layer and 1 neuron in the output layer for predicting pollution.
We use the Mean Absolute Error (MAE) loss function and the Adam version of stochastic gradient descent.
The model will be fit for 50 training epochs with a batch size of 72. 
At the end of the run both the training and test loss are plotted.

In [ ]:
# Sequential model is appropriate to design LSTM RNN
model = Sequential()
# train_X : 3D tensor (samples, lag_hours, feature)
# neurons =50
# single input_shape = (lag_hours, feature)
model.add(LSTM(50, input_shape=(?,?)))
model.add(Dense(1))  # 1 neuron in the output layer
model.compile(loss='mae', optimizer='adam')

# fit network
#verbose=0 will show nothing (silent)
#verbose=1 will show an animated progress bar
#verbose=2 will show the training progress for each epoch (shows loss and val_loss)

history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

#  print history
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('Loss function')
pyplot.legend(['train', 'test'])
pyplot.xlabel('iterations')
pyplot.show()
 

## Evaluate Model
After the model is fit, apply it to predict test data.
Compare the predictions (yhat) and the real data (test_y) as shown in Fig. 2. 

<img src="images/f2.png" style="width:350px;height:250px;">
<caption><center> **Fig. 1** : ** Test data prediction (the first 2000 samples)** </center></caption>

In [ ]:
#Apply the model to predict the test data
yhat = ?

# Compute the Mean Squared Error (MSE) on test data with function mean_squared_error 
#(ANSWER: Test mse = 0.001)
mse = ?


#Plot test_y and yhat on the same plot. 
#use pyplot from matplotlib
#Since test data has huge amount of samples (35037), the plot visibility is bad. 
#To improve it choose to plot only the fist 2000 samples.

?


In [ ]:
#Repeat the same as the previous cell but now with train data
?